In [68]:
import pandas as pd
import datetime

#api
import praw

#pre-processing
import nltk 
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import MWETokenizer 
from nltk import word_tokenize
import re
# import string
#from textblob import TextBlob
#from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
# nltk.download()

#analysis
from sklearn.decomposition import TruncatedSVD, NMF

In [3]:
reddit = praw.Reddit(client_id='oCxykEUclueAuA', client_secret='M8wVQw3nFxMsXHLGxurnTtkN-Mdvtg', user_agent='Crypto')

Version 7.0.0 of praw is outdated. Version 7.1.4 was released Sunday February 07, 2021.


#### Get top posts of all time

In [4]:
posts = []
ml_subreddit = reddit.subreddit('Cryptocurrency')

for post in list(ml_subreddit.top('all', limit=1000)):
    date = datetime.datetime.utcfromtimestamp(post.created_utc)
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, date])
    
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'date'])
print(posts)

                                                 title  score      id  \
0    Elon Musk reminds me of a rich billionaire who...  39166  lce6zm   
1    CryptoNick is deleting all of his BitConnect v...  26507  7r0ftz   
2    Reminder: Robinhood blocked several stocks fro...  26220  lfrslo   
3    I will tell you exactly what is going on here,...  20143  7vga1y   
4    Robinhood is launching a Crypto Trading app to...  19969  7sx5ze   
..                                                 ...    ...     ...   
927                                 We Are All Satoshi   1487  gqt4ze   
928  Hackers Donate Stolen Bitcoin Worth $20,000 to...   1491  jeowcu   
929  Venezuelan money so worthless they just use it...   1485  bl34bp   
930  Your Guide to Monero, and Why It Has Great Pot...   1488  7ra409   
931             Top 101 Coins Grouped by Usage/Purpose   1484  lgeots   

          subreddit                                                url  \
0    CryptoCurrency  https://www.reddit.com/r/Cry

#### Get top coments from each top post

In [5]:
top_comments = []
for i,d in zip(posts.id, posts.date):
    try:
        submission = reddit.submission(id=i)
        submission.comments.replace_more(limit=0)
        comments=list([(comment.body) for comment in submission.comments.list()])
        for x in comments[:20]:
            top_comments.append([x,d])
    except:
        comments=None
top_comments = pd.DataFrame(top_comments,columns=['comment', 'date'])
print(top_comments)

                                                 comment                date
0      If your life savings hinges on a Tweet from a ... 2021-02-04 12:02:16
1      I seriously don’t understand the obsession wit... 2021-02-04 12:02:16
2      Tbh u can't place all the blame on Elon tho, t... 2021-02-04 12:02:16
3      seriously, there's so much more relevant peopl... 2021-02-04 12:02:16
4      If I'm Elon, I put hundreds of millions into D... 2021-02-04 12:02:16
...                                                  ...                 ...
18616  What kept you from keeping Monero from digital... 2021-02-09 22:02:05
18617             Wow that‘s a good overview. Thank you. 2021-02-09 22:02:05
18618  Hi guys,\n\nI am from India and there is a pos... 2021-02-09 22:02:05
18619  I use nano as a store of value is it not a sto... 2021-02-09 22:02:05
18620  Vechain is not a supply chain logistics compan... 2021-02-09 22:02:05

[18621 rows x 2 columns]


## Cleaning

In [71]:
all_comments = ' '.join(top_comments['comment']) #concatenate comments column into one string

all_comments = re.sub(r'\([^)]*\)', '', all_comments) #remove links
all_comments = re.sub('\w*\d\w*', '', all_comments) #remove numbers
all_comments = re.sub(r'[^\w\s]', '', (all_comments))#remove punctuation
all_comments = all_comments.lower() #lowercase
all_comments = all_comments.replace('btc', 'bitcoin') #replace bitcoin acronym 

#create stop words list
stop = stopwords.words('english')
stop += ['crypto', 'moderator', 'subreddit', 'reddit' , 'question', 'concern', 'please', 'contact', 'im', 
        'people', 'gon', 'seems', 'something', 'year', 'like']
stop = set(stop)

mwe_tokenizer = MWETokenizer([('doge','coin'), ('bull','run'), ('market','cap'), ('pump','dump'),
                             ('stock','market'), ('flaired', 'inaccurately')])

words = mwe_tokenizer.tokenize(word_tokenize(all_comments))  # tokenize words

#lemmatize words
lemmatizer = nltk.stem.WordNetLemmatizer()
words = [(lemmatizer.lemmatize(w)) for w in words]

words = [w for w in words if w not in stop] #remove stop words

#create n grams
n = 3
bigrams = ngrams(words, n)

#count phrases
counter = Counter()
counter += Counter(bigrams)

for word, count in counter.most_common(30):
    print('%20s %i' % (" ".join(word), count))

       bot action wa 104
 action wa performed 104
wa performed automatically 104
flaired_inaccurately click bot 68
    click bot action 68
submission wa flaired_inaccurately 65
wa flaired_inaccurately click 65
summary auto generated 24
abstract history exchange 20
history exchange wallet 20
exchange wallet bias 20
wallet bias argument 20
bias argument argument 20
argument argument cryptowikis 20
argument cryptowikis policy 20
cryptowikis policy contribute 20
  auto generated bot 20
 generated bot meant 20
   bot meant replace 20
meant replace reading 20
replace reading original 20
reading original article 20
original article always 20
 article always dyor 20
  basic info website 19
      get rich quick 17
      dont even know 17
      word word word 17
policy contribute content 16
contribute content bot 16


In [62]:
# all_comments = ' '.join(top_comments['comment']) #concatenate comments column into one string

# all_comments = all_comments.lower() #lowercase
# all_comments = all_comments.replace('btc', 'bitcoin') #replace bitcoin acronym 
# all_comments = re.sub(r'\([^)]*\)', '', all_comments) #remove links

# mwe_tokenizer = MWETokenizer([('doge','coin')])

# words = mwe_tokenize.tokenize(TextBlob(all_comments).words)  # tokenize words


# words = [(re.sub('\w*\d\w*', '', w)) for w in words] #remove numbers
# words = [(re.sub(r'[^\w\s]', '', (w))) for w in words] #remove punctuation

# #create stop words list
# stop = stopwords.words('english')
# stop += ['crypto', 'moderator', 'subreddit', 'reddit' , 'question', 'concern', 'please', 'contact', 'im', ' ']
# stop = set(stop)

# #lemmatize words
# lemmatizer = nltk.stem.WordNetLemmatizer()
# words = [(lemmatizer.lemmatize(w)) for w in words]

# words = [w for w in words if w not in stop] #remove stop words

# #create n grams
# n = 2
# bigrams = ngrams(words, n)

# #count phrases
# counter = Counter()
# counter += Counter(bigrams)

# for word, count in counter.most_common(30):
#     print('%20s %i' % (" ".join(word), count))

## Vectorization

In [29]:
comments = list(top_comments['comment'])

#comments_cleaned = [re.sub('\w*\d\w*', '', i) for i in comments]

# set(stopwords.words('english'))
cv = CountVectorizer(stop_words=stop)
X= cv.fit_transform(comments_cleaned)
pd.DataFrame(X.toarray(), columns=cv.get_feature_names())

NameError: name 'comments_cleaned' is not defined

## Topic Modeling

#### Latent Semantic Analysis (LSA)

In [ ]:
lsa = TruncatedSVD(3)
doc_topic = lsa.fit_transform(X)
lsa.explained_variance_ratio_

In [ ]:
topic_word = pd.DataFrame(lsa.components_.round(3),
             index = ["component_1","component_2", "component_3"],
             columns = cv.get_feature_names())
topic_word

In [ ]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [ ]:
display_topics(lsa, cv.get_feature_names(), 5)

In [ ]:
Vt = pd.DataFrame(doc_topic.round(5),
             index = comments,
             columns = ["component_1","component_2", "component_3" ])
Vt

#### Non-Negative Matrix Factorization (NMF)

In [ ]:
nmf_model = NMF(3)
doc_topic = nmf_model.fit_transform(X)

In [ ]:
topic_word = pd.DataFrame(nmf_model.components_.round(3),
             index = ["component_1","component_2", "component_3"],
             columns = cv.get_feature_names())
topic_word

In [ ]:
display_topics(nmf_model, cv.get_feature_names(), 10)

In [ ]:
H = pd.DataFrame(doc_topic.round(5),
             index = comments,
             columns = ["component_1","component_2", "component_3" ])
H